# 1. Import and Install Dependencies

import matplotlib

In [1]:
import sklearn

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model for the people detection 
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

### Function to draw all the landmarks

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

### Application of the function already defined with the 4 different types of landmarks.

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
   
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    '''
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) '''

In [ ]:
mp_holistic.PoseLandmarks??

In [ ]:
mp_pose = mp.solutions.pose
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        try:
            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]
            print(shoulder,elbow,wrist)
            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)
        except :
            pass 

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
results

# 3. Extract Keypoint Values

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

### result variable is all the points saved by  cv2 after the detection by mediapipe

### Extract the key point and gives it a shape of a np array with 4 different variables each.

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    #lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    #rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose]) #, face, lh, rh

#possiamo semplicemnete togliere il concatenatee returnare pose

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
np.save('0', result_test)

In [ ]:
np.load('0.npy')

# 4. Setup Folders for Collection

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['curl', 'squat','flessioni','null'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length ????
sequence_length = 30

In [ ]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions: # loop over actions that are three for now 
        # Loop through sequences aka videos
        for sequence in range(no_sequences): ## loop over 30 videos per actions
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
            
                # Read feed
                ret, frame = cap.read() # create a single frame in which we capture frames
                
                # Make detections
                image, results = mediapipe_detection(frame, holistic) # detection of mediapipe
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results) 
                
                # NEW Apply wait logic
                if frame_num == 0: # first frame of the video 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),  ## only text advisor  
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)  
                    ## only text advisor  
                    cv2.putText(image, ' for {} n° {}'.format(action, sequence), (30,50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(5000)
                else: 
                    cv2.putText(image, ' for {}  n° {}'.format(action, sequence), (30,50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                 
                ## THE FUNCTION CORE STARTS FROM HERE : 
                # NEW Export keypoints
                keypoints = extract_keypoints(results) # call extract keypoints so we have to modify this
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) #join the path  
                np.save(npy_path, keypoints) # save the points in the path specified before

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2023-06-04 17:14:28.586503: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'curl': 0, 'squat': 1, 'flessioni': 2, 'null': 3}

In [11]:
## necessario per creare la parte del dataset utile
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(120, 30, 132)

In [12]:
np.array(labels).shape

(120,)

In [13]:
X = np.array(sequences)

In [14]:
X.shape

(120, 30, 132)

In [14]:
y = to_categorical(labels).astype(int)

In [16]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [15]:
#splitting test with train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40)

In [18]:
y_test.shape

(48, 4)

# 7. Build and Train LSTM Neural Network

In [16]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['curl', 'squat','flessioni','null'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [19]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,132)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

2023-06-04 17:15:21.519487: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
res = [.7, 0.2, 0.1]                                                                                               

In [21]:
actions[np.argmax(res)]

'curl'

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])

Epoch 1/300
3/3 [==============================] - 4s 58ms/step - loss: 1.3832 - categorical_accuracy: 0.3056
Epoch 2/300
3/3 [==============================] - 0s 45ms/step - loss: 1.3778 - categorical_accuracy: 0.3194
Epoch 3/300
3/3 [==============================] - 0s 40ms/step - loss: 1.3412 - categorical_accuracy: 0.3194
Epoch 4/300
3/3 [==============================] - 0s 42ms/step - loss: 1.3139 - categorical_accuracy: 0.3194
Epoch 5/300
3/3 [==============================] - 0s 42ms/step - loss: 1.1728 - categorical_accuracy: 0.5139
Epoch 6/300
3/3 [==============================] - 0s 39ms/step - loss: 0.8535 - categorical_accuracy: 0.6250
Epoch 7/300
3/3 [==============================] - 0s 40ms/step - loss: 0.9524 - categorical_accuracy: 0.5417
Epoch 8/300
3/3 [==============================] - 0s 41ms/step - loss: 0.8707 - categorical_accuracy: 0.6250
Epoch 9/300
3/3 [==============================] - 0s 39ms/step - loss: 0.8774 - categorical_accuracy: 0.5556
Epoch 10/3

3/3 [==============================] - 0s 59ms/step - loss: 4.6894 - categorical_accuracy: 0.3889
Epoch 148/300
3/3 [==============================] - 0s 63ms/step - loss: 1.4687 - categorical_accuracy: 0.4028
Epoch 149/300
3/3 [==============================] - 0s 66ms/step - loss: 1.8194 - categorical_accuracy: 0.4444
Epoch 150/300
3/3 [==============================] - 0s 57ms/step - loss: 0.8361 - categorical_accuracy: 0.6528
Epoch 151/300
3/3 [==============================] - 0s 61ms/step - loss: 0.8651 - categorical_accuracy: 0.6667
Epoch 152/300
3/3 [==============================] - 0s 77ms/step - loss: 0.8771 - categorical_accuracy: 0.4861
Epoch 153/300
3/3 [==============================] - 0s 48ms/step - loss: 0.8689 - categorical_accuracy: 0.5278
Epoch 154/300
3/3 [==============================] - 0s 74ms/step - loss: 0.8581 - categorical_accuracy: 0.6806
Epoch 155/300
3/3 [==============================] - 0s 50ms/step - loss: 0.8525 - categorical_accuracy: 0.6806
Epoch 

3/3 [==============================] - 0s 61ms/step - loss: 0.2411 - categorical_accuracy: 0.8889
Epoch 294/300
3/3 [==============================] - 0s 66ms/step - loss: 0.2414 - categorical_accuracy: 0.8889
Epoch 295/300
3/3 [==============================] - 0s 65ms/step - loss: 0.2523 - categorical_accuracy: 0.8750
Epoch 296/300
3/3 [==============================] - 0s 66ms/step - loss: 0.2057 - categorical_accuracy: 0.9167
Epoch 297/300
3/3 [==============================] - 0s 74ms/step - loss: 0.2292 - categorical_accuracy: 0.9028
Epoch 298/300
3/3 [==============================] - 0s 66ms/step - loss: 0.3555 - categorical_accuracy: 0.8333
Epoch 299/300
3/3 [==============================] - 0s 60ms/step - loss: 0.7557 - categorical_accuracy: 0.7639
Epoch 300/300
3/3 [==============================] - 0s 58ms/step - loss: 0.3089 - categorical_accuracy: 0.8611


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            50432     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 205,028
Trainable params: 205,028
Non-trai

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[2])]

In [ ]:
actions[np.argmax(y_test[3])]

# 9. Save Weights

### Save the weights

In [ ]:
model.save('Actions Databases/action.h5')

### Deleate Model

In [ ]:
del model

### !! Load the model !!

In [ ]:
model.load_weights('Actions Databases/action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [25]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [26]:
yhat = model.predict(X_test)

2/2 [==============================] - 1s 41ms/step


In [28]:
yhat

array([[2.01851532e-01, 4.12050992e-01, 1.66362047e-01, 2.19735458e-01],
       [9.16050121e-05, 3.34232391e-06, 9.11101873e-04, 9.98993933e-01],
       [5.11700091e-05, 1.40643272e-06, 1.81251569e-04, 9.99766171e-01],
       [2.01538116e-01, 4.60421056e-01, 1.54974550e-01, 1.83066338e-01],
       [6.94607422e-02, 8.17525327e-01, 6.14703260e-02, 5.15435785e-02],
       [1.10468671e-01, 6.84816897e-01, 1.19302437e-01, 8.54119658e-02],
       [2.90405287e-13, 5.80548115e-14, 9.99997616e-01, 2.33307105e-06],
       [2.14212071e-20, 6.23121618e-21, 1.00000000e+00, 6.64188038e-10],
       [6.43710345e-02, 8.42990875e-01, 5.02154976e-02, 4.24226783e-02],
       [1.76000074e-01, 5.25843322e-01, 1.28007293e-01, 1.70149297e-01],
       [2.45585828e-03, 2.12062546e-03, 9.51019585e-01, 4.44040075e-02],
       [1.86017498e-01, 4.77356851e-01, 1.98552772e-01, 1.38072938e-01],
       [9.54805985e-02, 7.49435127e-01, 8.19315836e-02, 7.31527433e-02],
       [7.84440413e-02, 7.78861165e-01, 8.51775855e

In [29]:
X_test

array([[[ 0.50573695,  0.256134  , -0.16374241, ...,  0.82333291,
          0.03644178,  0.99533421],
        [ 0.51024157,  0.25656271, -0.19059637, ...,  0.8355701 ,
         -0.11533155,  0.99481273],
        [ 0.51204926,  0.25653902, -0.20504794, ...,  0.83964396,
         -0.08652759,  0.99416417],
        ...,
        [ 0.51323378,  0.25985882, -0.1819841 , ...,  0.84129071,
         -0.1200253 ,  0.97807336],
        [ 0.5138737 ,  0.25931451, -0.18307537, ...,  0.84128332,
         -0.125076  ,  0.97889453],
        [ 0.5141083 ,  0.25857228, -0.18698326, ...,  0.84126085,
         -0.12569842,  0.97972888]],

       [[ 0.48559442,  0.22580031, -0.26524064, ...,  0.87959713,
          0.04692725,  0.95776087],
        [ 0.48451823,  0.23219556, -0.21193132, ...,  0.85493106,
          0.15112171,  0.9593485 ],
        [ 0.48372969,  0.23519351, -0.22106762, ...,  0.85000092,
          0.17003702,  0.96154469],
        ...,
        [ 0.48421264,  0.23620681, -0.19209215, ...,  

In [ ]:
y_test

In [27]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [28]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[33,  1],
        [11,  3]],

       [[23, 11],
        [ 5,  9]],

       [[35,  0],
        [ 1, 12]],

       [[36,  5],
        [ 0,  7]]])

In [29]:
accuracy_score(ytrue, yhat)

0.6458333333333334

# 11. Test in Real Time

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245),(45,110,160)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence.reverse()

In [ ]:
len(sequence)

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

### Test
1.Importare funzioni **mp_holistic,**draw_styled landmarks,extract_keypoints

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks  
       
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 0: # first frame of the video 
            cv2.putText(image, 'STARTING COLLECTION', (120,200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA) 
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))